In [ ]:
#-----------------------------------------------
# @CopyRight - Hasini Nakulugamuwa Gamage
# Federation University Australia
#-----------------------------------------------

import numpy as np
import xlrd
import math
import networkx as nx
import matplotlib.pyplot as plt
from pyitlib import discrete_random_variable as drv
from sklearn.preprocessing import Binarizer
import statistics
import pandas as pd

#-----MRMR Implementation------------------------
def MutualInformationBased_InteractionSelector(a, a2, S, S1, k, x, maxValc):
    while k < 10:
        maxGSVal = []
        itemArr = []
        pccValArr = []
        if not S:
            MIArr = []
            pccValArr1 = []
            regInit1 = -1
            for b in range(N):
                b2 = []
                for e1 in range(nt - 1):
                    b2.append(int(x[b,e1]))
                
                X = np.array(a2)
                Y = np.array(b2)
                valc = drv.information_mutual(X, Y, cartesian_product=False)
                if valc > 0:
                    MIArr.append((valc,b))
                    pccVal = np.corrcoef(X,Y)
                    if pccVal[0,1] < 0:
                        pccValArr1.append(b)
                    if valc >= maxValc:
                        maxValc = valc
                        regInit = b
                        if pccVal[0,1] < 0:
                            regInit1 = b 
            if maxValc == 0:
                S = []
                S1 = []
                CMIArr = []
                PCCArr = []
                return S, S1, CMIArr, PCCArr
            S.append(regInit)
            S1.append(regInit1)
            k = k + 1
            
        else:
            for b in range(N):
                b2 = []
                d2 = []
                TotalVal = 0
                if b not in S:
                    for e1 in range(nt - 1):
                        b2.append(int(x[b,e1]))
                    X = np.array(a2)
                    Y = np.array(b2)
                    valc = drv.information_mutual(X, Y, cartesian_product=False)
                    if valc > 0:
                        pccVal = np.corrcoef(X,Y)
                        for element1 in range(nt - 1):
                            tempVal = x[b, element1 + 1]
                            x2[b, element1] = tempVal
                            d2.append(int(x2[b, element1]))
                        for element in S:
                            c2 = []
                            for gs in range(nt - 1):
                                c2.append(int(x[element,gs]))
                            SVal = drv.information_mutual(d2, c2, cartesian_product=False)
                            TotalVal = TotalVal + SVal
                        GSVal = valc - (TotalVal/k)
                        maxGSVal.append(GSVal)
                        itemArr.append(b)
                        pccValArr.append(pccVal[0,1])
            if not maxGSVal:
                CMIArr = MIArr.copy()
                PCCArr = pccValArr1.copy() 
                CMIArr.sort(reverse=True)
                return S, S1, CMIArr, PCCArr
            
            max_value = max(maxGSVal)
            max_index = maxGSVal.index(max_value)
            nextItem = itemArr[max_index]
            if pccValArr[max_index] < 0:
                nextItem1 = nextItem
                S1.append(nextItem1)
            S.append(nextItem)
            k = k + 1
    CMIArr = MIArr.copy()
    PCCArr = pccValArr1.copy() 
    CMIArr.sort(reverse=True)
    return S, S1, CMIArr, PCCArr

#---------Boolean variable state update function-------------------------------------------------------------------
def Search_Update_Rule(Sn, S1n, x, t, sign):
    if sign == 'c':
        for de in range (len(Sn)):
            deVal = int(Sn[de])     
            if de == 0:
                if deVal in S1n:
                    fvaln = int(x[deVal,t])
                    if fvaln == 0:
                        fvaln = 1
                    else:
                        fvaln = 0
                else:
                    fvaln = int(x[deVal,t])
            else:
                if deVal in S1n:
                    valn = x[deVal,t]
                    if valn == 0:
                        valn = 1
                    else:
                        valn = 0
                else:
                    valn = int(x[deVal,t])
                if valn == 0:
                    Boolvaln = False
                else:
                    Boolvaln = True
                if fvaln == 0:
                    Boolfvaln = False
                else:
                    Boolfvaln = True
                if Boolvaln and Boolfvaln:
                    fvaln = 1
                else:
                    fvaln = 0
        lastVal = fvaln
    if sign == 'd':
        for de in range (len(Sn)):
            deVal = int(Sn[de])     
            if de == 0:
                if deVal in S1n:
                    fvaln = int(x[deVal,t])
                    if fvaln == 0:
                        fvaln = 1
                    else:
                        fvaln = 0
                else:
                    fvaln = int(x[deVal,t])
            else:
                if deVal in S1n:
                    valn = x[deVal,t]
                    if valn == 0:
                        valn = 1
                    else:
                        valn = 0
                else:
                    valn = int(x[deVal,t])
                if valn == 0:
                    Boolvaln = False
                else:
                    Boolvaln = True
                if fvaln == 0:
                    Boolfvaln = False
                else:
                    Boolfvaln = True
                if Boolvaln or Boolfvaln:
                    fvaln = 1
                else:
                    fvaln = 0
        lastVal = fvaln
    return lastVal

#-----------SWAP operation-----------------------------------------------------------------------------------------------
def SWAP_Function(a, Sa, Wa, PccN, x, maxDynamicAccuaracy, updateRule):
    S_c = Sa.copy()
    W_c = Wa.copy()
    vmaxDynamicAccuaracy = maxDynamicAccuaracy
    for Wi in range(len(W_c)):
        for Sj in range(len(S_c)):
            TempW = W_c[Wi]
            TempS = S_c[Sj]
            S_c[Sj] = TempW
            W_c[Wi] = TempS
            cgeneAccu = 0
            dgeneAccu = 0
            for signEl in sign:
                if signEl == 'c':
                    for cts in range(nt - 1):
                        v0 = int(x[a,cts + 1])
                        csurVal = Search_Update_Rule(S_c, PccN, x, cts, signEl)
                        if v0 == csurVal:
                            cgeneAccu = cgeneAccu + 1
                        else:
                            cgeneAccu = cgeneAccu
                    cTgeneAccu = cgeneAccu/(nt-1)
                    ConGeneAccu = cTgeneAccu
                if signEl == 'd':
                    for dts in range(nt - 1):
                        v0 = int(x[a,dts + 1])
                        dsurVal = Search_Update_Rule(Sn, S1n, x, dts, signEl)
                        if v0 == dsurVal:
                            dgeneAccu = dgeneAccu + 1
                        else:
                            dgeneAccu = dgeneAccu
                    dTgeneAccu = dgeneAccu/(nt-1)
                    DidjGeneAccu = dTgeneAccu
            if ConGeneAccu >= DidjGeneAccu:
                c_maxDynamicAccuaracy = ConGeneAccu
                update_rule = 'c'
            else:
                c_maxDynamicAccuaracy = DidjGeneAccu
                update_rule = 'd'
            if c_maxDynamicAccuaracy > vmaxDynamicAccuaracy:
                Sa = S_c.copy()
                Wa = W_c.copy()
                vmaxDynamicAccuaracy = c_maxDynamicAccuaracy
                updateRule = update_rule
                if vmaxDynamicAccuaracy == 1.0:
                    return a, vmaxDynamicAccuaracy, updateRule, Sa, Wa
            else:
                S_c = Sa.copy()
                W_c = Wa.copy()
    return a, vmaxDynamicAccuaracy, updateRule, Sa, Wa

#-------------------------------------------------------------------------------------------------------
#------Main Starts Here
#-------------------------------------------------------------------------------------------------------

loc1 = %path% #-- File path for dataset
wb1 = xlrd.open_workbook(loc1)
sheet1 = wb1.sheet_by_index(0)

N = %Gene#%
nt = %TimeSamples#%
xinit = np.zeros((N, nt))
x = np.zeros((N, nt))
x1 = np.zeros((N, nt - 1))
x2 = np.zeros((N, nt - 1))
RN = [[1], [], [1], [8,2], [8,7,2], [2], [2,7,9], [], [], []] #---- Target gene network details
TP = 0
FP = 0
FN = 0
TN = 0
meanArr = []
TotalNetworkDynamicAccuracy = 0

for i in range(N):
    for j in range(nt):
        value = sheet1.cell_value(j, i)   
        xinit[i,j] = value
        
file_name = %path% #-- File path for dataset
my_sheet = 'Sheet1'

pd_full_data_set = pd.read_excel(file_name, sheet_name = my_sheet)
av_col = pd_full_data_set.mean(axis=0)

for binz in range(N):
    for condi in range(nt):
        if xinit[binz,condi] >= av_col[binz]:
            x[binz,condi] = 1
        else:
            x[binz,condi] = 0

#------------------------------------------------------------------------------
print("================ Discretized Gene Expression Dataset =================")
print(x)
print("======================================================================")

G = nx.DiGraph()
G1 = nx.DiGraph()

#---------Gene wise operation-----------
for a in range(N):
    item = []
    a2 = []
    S = []
    S1 = []
    k = 0
    maxValc = 0
    geneEntropy = 1
    maxDynamicAccuaracy = 0
    Inter_Arr = []
    negativeGeneArr = []
    for tx in range(nt - 1):
        tVal = x[a,tx + 1]
        x1[a, tx] = tVal
        a2.append(int(x1[a, tx]))
    geneEntropy = drv.entropy(a2)
    print("Gene Entropy of Gene : " + str(a) + " is " + str(geneEntropy))
    if geneEntropy == 0:
        print("No regulatory Genes")
        TotalNetworkDynamicAccuracy = TotalNetworkDynamicAccuracy + 1.0
        if len(RN[a]) > 0:
                FN = FN + len(RN[a])
                for ra1 in RN[a]:
                    G1.add_edge(ra1,a,color='b')
    else:
        SArr, S1Arr, Inter_Arr, PccoefiArr = MutualInformationBased_InteractionSelector(a, a2, S, S1, k, x, maxValc)
        if not SArr:
            print("************************ Gene No " + str(a) + " ********************************")
            print("**Regulatory Gene Set for Target Gene : " + str(a) + "**")
            print("No regulatory Genes")
            TotalNetworkDynamicAccuracy = TotalNetworkDynamicAccuracy + 1.0
            if len(RN[a]) > 0:
                FN = FN + len(RN[a])
                for ra1 in RN[a]:
                    G1.add_edge(ra1,a,color='b')
        else: 
            print("************************ Gene No " + str(a) + " ********************************")
            print("**Regulatory Gene Set for Target Gene before SWAP : " + str(a) + "**")
            print(SArr)
            print("*********************************************************************")
            inSize = 0
            in2Arr = []
            sort2ArrS = []
            sort2ArrW = []
            for inArr in Inter_Arr:
                in2Arr.append(Inter_Arr[inSize][1])
                inSize = inSize + 1
            for sortArr in in2Arr:
                if sortArr in SArr:
                    sort2ArrS.append(sortArr)
                else:
                    sort2ArrW.append(sortArr)
    
            Sn = sort2ArrS.copy()
            PccN = PccoefiArr.copy()
            deadVal = -1
            if deadVal in S1Arr:
                S1Arr.remove(-1)
            S1n = S1Arr.copy()
            negGenesTest = []
            for InterT in range(N):
                c00 = 0
                c01 = 0
                for interTypet in range(nt - 1):
                    if x[InterT,interTypet] == x[a,interTypet + 1]:
                        c00 = c00 + 1
                    else:
                        c01 = c01 + 1
                if c00 >= c01:
                    inType = "POSITIVE"
                else:
                    negGenesTest.append(InterT)
                    inType = "NEGATIVE"
            #--------------------------------------------------------------------------------------------
            cgeneAccu = 0
            dgeneAccu = 0
            sign = ['c', 'd']
            for signEl in sign:
                if signEl == 'c':
                    for cts in range(nt - 1):
                        v0 = int(x[a,cts + 1])
                        csurVal = Search_Update_Rule(Sn, negGenesTest, x, cts, signEl)
                        if v0 == csurVal:
                            cgeneAccu = cgeneAccu + 1
                        else:
                            cgeneAccu = cgeneAccu
                    cTgeneAccu = cgeneAccu/(nt-1)
                    ConGeneAccu = cTgeneAccu
                if signEl == 'd':
                    for dts in range(nt - 1):
                        v0 = int(x[a,dts + 1])
                        dsurVal = Search_Update_Rule(Sn, negGenesTest, x, dts, signEl)
                        if v0 == dsurVal:
                            dgeneAccu = dgeneAccu + 1
                        else:
                            dgeneAccu = dgeneAccu
                    dTgeneAccu = dgeneAccu/(nt-1)
                    DidjGeneAccu = dTgeneAccu
            if ConGeneAccu >= DidjGeneAccu:
                maxDynamicAccuaracy = ConGeneAccu
                updateRule = 'c'
                print("Selected Dynamic Gene Accuracy of Gene " + str(a) + " : " + str(ConGeneAccu))
                print("Selected Update Rule : Conjunction (AND Operator)")
            else:
                maxDynamicAccuaracy = DidjGeneAccu
                updateRule = 'd'
                print("Selected Dynamic Gene Accuracy of Gene " + str(a) + " : " + str(DidjGeneAccu))
                print("Selected Update Rule : Disjunction (OR Operator)")
            if maxDynamicAccuaracy != 1.0:
                GeneNo, maxDynamicAccuaracy, updateRule, newS, newW = SWAP_Function(a, sort2ArrS, sort2ArrW, negGenesTest, x, maxDynamicAccuaracy, updateRule)
            else:
                GeneNo = a
                newS = sort2ArrS
                newW = sort2ArrW
            for negativeGene in newS:
                if negativeGene in negGenesTest:
                    negativeGeneArr.append(negativeGene)
            print("++++++++++++++++++++++++++++++++++++++++++++++++++++")
            print("*********** After SWAP ***********")
            print("Regulatory Genes : " + str(newS))
            for drawG in newS:
                if drawG in S1n:
                    G.add_edge(a,drawG,color='r')
                else:
                    G.add_edge(a,drawG,color='g')
            print("Genes with NOT rule : " + str(negativeGeneArr))
            print("Genes negatively identified with PCC : " + str(S1n))
            print("Non - Regulatory Genes : " + str(newW))
            print("Update Rule : " + str(updateRule))
            print("Gene - wise Dymanic Accuracy : " + str(maxDynamicAccuaracy))
            print("++++++++++++++++++++++++++++++++++++++++++++++++++++")
            
            TotalNetworkDynamicAccuracy = TotalNetworkDynamicAccuracy + maxDynamicAccuaracy
            
            for SAEl in newS:
                if SAEl in RN[a]:
                    TP = TP + 1
                    if SAEl in S1n:
                        G1.add_edge(SAEl,a,color='r')
                    else:
                        G1.add_edge(SAEl,a,color='g')
                else:
                    FP = FP + 1
                    if SAEl in S1n:
                        G1.add_edge(SAEl,a,color='b')  
                    else:
                        G1.add_edge(SAEl,a,color='b')
                        
            for RNEl in RN[a]:
                if RNEl not in newS:
                    FN = FN + 1
                    G1.add_edge(RNEl,a,color='y')
                        
TotalNetworkDynamicAccuracy_val = TotalNetworkDynamicAccuracy/N
print("#################################################################################")
print("Total Network Dynamic Accuracy : " + str(round(TotalNetworkDynamicAccuracy_val, 4)))   
print("#################################################################################")
print("True Positive : " + str(TP))
print("False Positive : " + str(FP))
print("False Negative : " + str(FN))

TN = int(N*N) - (TP + FP + FN)
print("True Negative : " + str(TN))

Precision = TP/(TP + FP)
Specificity = TN/(TN + FP)
Recall = TP/(TP + FN)
SA = (TP + TN)/(TP + FP + FN + TN)

print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("Precision : " + str(round(Precision, 4)))   
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("Specificity : " + str(round(Specificity, 4)))   
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("Recall : " + str(round(Recall, 4)))   
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print("#################################################################################")
print("Total Network Stuctural Accuracy : " + str(SA))   
print("#################################################################################")

#------------------------------------------------------------------------
colors = nx.get_edge_attributes(G1,'color').values()

pos = nx.circular_layout(G1)
nx.draw_networkx_labels(G1, pos)
#--------Draw nodes and edges
nx.draw_networkx_nodes(G1, pos, node_color="b")
nx.draw_networkx_edges(
    G1, pos, edge_color=colors,
    connectionstyle="arc3,rad=0.1"
)